In [1]:
# ignore warnings
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
#imports
from pathlib import Path
import datetime as dt
import hvplot.pandas
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

# Inflation Datasets

### Lat/Lon CSV Cleaning

In [3]:
# name csv path
world_geo_path = Path("world_geo_data.csv")

# create dataframe from csv
world_geo_df =pd.read_csv(world_geo_path)

world_geo_df.head()

,country_code,latitude,longitude,country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state
0,AD,42.546245,1.601554,Andorra,AK,63.588753,-154.493062,Alaska
1,AE,23.424076,53.847818,United Arab Emirates,AL,32.318231,-86.902298,Alabama
2,AF,33.939110,67.709953,Afghanistan,AR,35.201050,-91.831833,Arkansas
3,AG,17.060816,-61.796428,Antigua and Barbuda,AZ,34.048928,-111.093731,Arizona
4,AI,18.220554,-63.068615,Anguilla,CA,36.778261,-119.417932,California


In [4]:
# rename columns for clarity and consistency
world_geo_df.rename(columns={"country_code": "alpha-2"}, inplace=True)

# drop unnecessary columns
world_geo_df.drop(labels=["usa_state_code",
    "usa_state_latitude",
    "usa_state_longitude",
    "usa_state"
    ], axis=1, inplace=True)

world_geo_df.head()


,alpha-2,latitude,longitude,country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [5]:
# check for nulls
nulls = world_geo_df.isnull().any(axis=1)
null_rows = world_geo_df[nulls]
display(null_rows)

,alpha-2,latitude,longitude,country
156,NaN,-22.95764,18.49041,Namibia
226,UM,NaN,NaN,U.S. Minor Outlying Islands


In [6]:
# fix alpha-2 for Namibia
world_geo_df.at[156, "alpha-2"] = "NA"

# check
print(world_geo_df.at[156, "alpha-2"])

# delete row for US Minor Outlying Islands because their geography is not bound by lon/lat
world_geo_df.drop(226, inplace=True)

# check
nulls = world_geo_df.isnull().any(axis=1)
null_rows = world_geo_df[nulls]
display(null_rows)


NA


,alpha-2,latitude,longitude,country


In [7]:
# check data types
world_geo_df.dtypes

alpha-2       object
latitude     float64
longitude    float64
country       object
dtype: object

### World Inflation CSV Cleaning

In [8]:
# name csv path
world_inflation_path = Path("world_inflation_data.csv")

# create dataframe from csv, skipping blank rows and naming the header
world_inflation_df =pd.read_csv(world_inflation_path, header=2, skiprows=[0, 1])

world_inflation_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.372065,0.421441,0.474764,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,5.750981,5.370290,5.245878,6.571396,6.399343,4.720811,4.653665,7.321106,6.824727,10.773751
2,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,7.385772,4.673996,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN,NaN
3,Africa Western and Central,AFW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,2.439201,1.768436,2.130817,1.487416,1.764635,1.784050,1.760112,2.437609,3.653533,7.967574
4,Angola,AGO,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,8.777814,7.280387,9.353840,30.698958,29.842578,19.630594,17.079704,22.271564,25.754266,NaN


In [9]:
# drop unnecessary columns
world_inflation_df.drop(columns=["Indicator Name", "Indicator Code"], inplace=True)

# rename country_code column for clarity and consistency
world_inflation_df.rename(columns={"Country Code": "alpha-3", "Country Name":"country"}, inplace=True)

world_inflation_df.head()

,country,alpha-3,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.372065,0.421441,0.474764,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.750981,5.370290,5.245878,6.571396,6.399343,4.720811,4.653665,7.321106,6.824727,10.773751
2,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.385772,4.673996,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN,NaN
3,Africa Western and Central,AFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.439201,1.768436,2.130817,1.487416,1.764635,1.784050,1.760112,2.437609,3.653533,7.967574
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.777814,7.280387,9.353840,30.698958,29.842578,19.630594,17.079704,22.271564,25.754266,NaN


In [10]:
# check data types
world_inflation_df.dtypes

country     object
alpha-3     object
1960       float64
1961       float64
1962       float64
            ...   
2018       float64
2019       float64
2020       float64
2021       float64
2022       float64
Length: 65, dtype: object

### ISO2/ISO3 CSV Cleaning

In [11]:
# path
country_info_path = Path("country_info.csv")

# create dataframe from csv
country_info_df = pd.read_csv(country_info_path)

country_info_df.head()


,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [12]:
# keep necessary columns
country_info_df = country_info_df[["name", "alpha-2", "alpha-3"]]

# change column names for consistency and clarity
country_info_df.rename(columns={"name":"country"}, inplace=True)

country_info_df.head()

,country,alpha-2,alpha-3
0,Afghanistan,AF,AFG
1,Åland Islands,AX,ALA
2,Albania,AL,ALB
3,Algeria,DZ,DZA
4,American Samoa,AS,ASM


In [13]:
# check for nulls
nulls = country_info_df.isnull().any(axis=1)
null_rows = country_info_df[nulls]
display(null_rows)

,country,alpha-2,alpha-3
153,Namibia,NaN,NAM


In [14]:
# fix issue with Namibia alpha-2 (NaN)
country_info_df.at[153, "alpha-2"] = "NA"
# check
print(country_info_df.at[153, "alpha-2"])

NA


In [15]:
# check datatypes
country_info_df.dtypes

country    object
alpha-2    object
alpha-3    object
dtype: object

### Combining 3 Dataframes

In [16]:
# view dataframes together
display(country_info_df.head())
display(world_inflation_df.head())
display(world_geo_df.head())

,country,alpha-2,alpha-3
0,Afghanistan,AF,AFG
1,Åland Islands,AX,ALA
2,Albania,AL,ALB
3,Algeria,DZ,DZA
4,American Samoa,AS,ASM


,country,alpha-3,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.372065,0.421441,0.474764,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.750981,5.370290,5.245878,6.571396,6.399343,4.720811,4.653665,7.321106,6.824727,10.773751
2,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.385772,4.673996,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN,NaN
3,Africa Western and Central,AFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.439201,1.768436,2.130817,1.487416,1.764635,1.784050,1.760112,2.437609,3.653533,7.967574
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.777814,7.280387,9.353840,30.698958,29.842578,19.630594,17.079704,22.271564,25.754266,NaN


,alpha-2,latitude,longitude,country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [17]:
# merge dataframes (merge instead of join because of dissimilar indexes)
combined_df = country_info_df.merge(world_geo_df, how="inner", on="alpha-2")
combined_df = combined_df.merge(world_inflation_df, how="inner", on="alpha-3")
combined_df

,country_x,alpha-2,alpha-3,latitude,longitude,country_y,country,1960,1961,1962,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,AF,AFG,33.939110,67.709953,Afghanistan,Afghanistan,NaN,NaN,NaN,...,7.385772,4.673996,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN,NaN
1,Albania,AL,ALB,41.153332,20.168331,Albania,Albania,NaN,NaN,NaN,...,1.937621,1.625865,1.896174,1.275432,1.986661,2.028060,1.411091,1.620887,2.041472,6.725203
2,Algeria,DZ,DZA,28.033886,1.659626,Algeria,Algeria,NaN,NaN,NaN,...,3.254239,2.916927,4.784447,6.397695,5.591116,4.269990,1.951768,2.415131,7.226063,9.265516
3,American Samoa,AS,ASM,-14.270972,-170.132217,American Samoa,American Samoa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,AD,AND,42.546245,1.601554,Andorra,Andorra,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,Virgin Islands (British),VG,VGB,18.420695,-64.639968,British Virgin Islands,British Virgin Islands,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,Virgin Islands (U.S.),VI,VIR,18.335765,-64.896335,U.S. Virgin Islands,Virgin Islands (U.S.),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,Yemen,YE,YEM,15.552727,48.516388,Yemen,"Yemen, Rep.",NaN,NaN,NaN,...,10.968442,8.104726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,Zambia,ZM,ZMB,-13.133897,27.849332,Zambia,Zambia,NaN,NaN,NaN,...,6.977676,7.806876,10.110593,17.869730,6.577312,7.494572,9.150316,15.733060,22.020768,10.993204


In [18]:
# drop duplicate columns
combined_df.drop(columns= ["country_y", "country", "alpha-2", "alpha-3"], inplace=True)

# rename country_x
combined_df.rename(columns={"country_x":"country"}, inplace=True)

# assign index
combined_df.set_index("country", inplace=True)

# sort by index
combined_df.sort_index(inplace=True)

# check
combined_df


,latitude,longitude,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,33.939110,67.709953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.385772,4.673996,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN,NaN
Albania,41.153332,20.168331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.937621,1.625865,1.896174,1.275432,1.986661,2.028060,1.411091,1.620887,2.041472,6.725203
Algeria,28.033886,1.659626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.254239,2.916927,4.784447,6.397695,5.591116,4.269990,1.951768,2.415131,7.226063,9.265516
American Samoa,-14.270972,-170.132217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,42.546245,1.601554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Virgin Islands (British),18.420695,-64.639968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Virgin Islands (U.S.),18.335765,-64.896335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,15.552727,48.516388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.968442,8.104726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
combined_df.dtypes

latitude     float64
longitude    float64
1960         float64
1961         float64
1962         float64
              ...   
2018         float64
2019         float64
2020         float64
2021         float64
2022         float64
Length: 65, dtype: object

In [20]:
# save as csv
combined_df.to_csv("combined_inflation_cleaned_data.csv")

# I Bonds Datasets

### cleaning data

In [21]:
# name csv path
ibonds_path = Path("US_Treasury_Series_I_Bond_Historical_Rates.csv")

# read csv
ibonds_df =pd.read_csv(ibonds_path)

ibonds_df.head()

,Date the rate was set,Fixed rate for bonds issued in the six months after that date,Inflation rate for all I bonds issued for six months (starting in that bond's next interest start month)
0,November 1 2022,0.4,3.24
1,May 1 2022,0.0,4.81
2,November 1 2021,0.0,3.56
3,May 1 2021,0.0,1.77
4,November 1 2020,0.0,0.84


In [22]:
# rename columns
ibonds_df.rename(columns={"Date the rate was set":"date", "Fixed rate for bonds issued in the six months after that date":"fixed rate", "Inflation rate for all I bonds issued for six months (starting in that bond's next interest start month)":"six-month inflation rate"}, inplace=True)

# drop nulls
ibonds_df.dropna(axis=0, inplace=True)

# correct error in start dates
ibonds_df["date"][41] = "May 1 2002"
ibonds_df["date"][42] = "November 1 2001"

# add missing data
# source: https://www.treasurydirect.gov/savings-bonds/i-bonds/i-bonds-interest-rates/
new_rows = {"date":["November 1 2023", "May 1 2023"],
    "fixed rate":["1.3", ".90"],
    "six-month inflation rate":["1.97", "1.69"]
}

# create new df
new_rows_df = pd.DataFrame(new_rows)

# concat dataframes
ibonds_combined_df = pd.concat([new_rows_df, ibonds_df])
ibonds_combined_df

# set date to datetime
ibonds_combined_df["date"] = pd.to_datetime(ibonds_combined_df["date"], format="%B %d %Y")

# reset index
ibonds_combined_df.reset_index(inplace=True)

# order ascending
ibonds_combined_df.sort_index(inplace=True, ascending=False)

# set date back as string
ibonds_combined_df["date"] = pd.to_datetime(ibonds_combined_df["date"]).dt.strftime("%m-%Y")

# set index
ibonds_combined_df.set_index("date", inplace=True)

# drop "index"
ibonds_combined_df.drop(columns=["index"], inplace=True)

# set dtypes to correct the manually entered data
ibonds_combined_df["fixed rate"] = ibonds_combined_df["fixed rate"].astype(float)
ibonds_combined_df["six-month inflation rate"] = ibonds_combined_df["six-month inflation rate"].astype(float)

# export cleaned df to csv for other uses
ibonds_combined_df.to_csv("ibonds_cleaned_data.csv")

ibonds_combined_df.head()

print(ibonds_combined_df)

         fixed rate  six-month inflation rate
date                                         
09-1998         3.4                      0.62
11-1998         3.3                      0.86
05-1999         3.3                      0.86
11-1999         3.4                      1.76
05-2000         3.6                      1.91
11-2000         3.4                      1.52
05-2001         3.0                      1.44
11-2001         2.0                      1.19
05-2002         2.0                      0.28
11-2002         1.6                      1.23
05-2003         1.1                      1.77
11-2003         1.1                      0.54
05-2004         1.0                      1.19
11-2004         1.0                      1.33
05-2005         1.2                      1.79
11-2005         1.0                      2.85
05-2006         1.4                      0.50
11-2006         1.4                      1.55
05-2007         1.3                      1.21
11-2007         1.2               

# Consumer Price Index Dataset

In [23]:
#### all urban consumer data seasonally adjusted ####
# name csv path
cpi_cpiu_path = Path("cpiu_data.csv")

# read csv
cpi_cpiu_data = pd.read_csv(cpi_cpiu_path)

cpi_cpiu_data.head()

,Series ID,Year,Period,Label,Value,1-Month % Change
0,CUSR0000SA0,1998,M01,1998 Jan,162.0,0.1
1,CUSR0000SA0,1998,M02,1998 Feb,162.0,0.0
2,CUSR0000SA0,1998,M03,1998 Mar,162.0,0.0
3,CUSR0000SA0,1998,M04,1998 Apr,162.2,0.1
4,CUSR0000SA0,1998,M05,1998 May,162.6,0.2


In [24]:
# drop unnecessary columns
cpi_cpiu_data.drop(columns=["Series ID"], inplace=True)

cpi_cpiu_data

,Year,Period,Label,Value,1-Month % Change
0,1998,M01,1998 Jan,162.000,0.1
1,1998,M02,1998 Feb,162.000,0.0
2,1998,M03,1998 Mar,162.000,0.0
3,1998,M04,1998 Apr,162.200,0.1
4,1998,M05,1998 May,162.600,0.2
...,...,...,...,...,...
306,2023,M07,2023 Jul,304.348,0.2
307,2023,M08,2023 Aug,306.269,0.6
308,2023,M09,2023 Sep,307.481,0.4
309,2023,M10,2023 Oct,307.619,0.0


In [25]:
# convert to datetime
cpi_cpiu_data["Label"] = pd.to_datetime(cpi_cpiu_data["Label"], format="%Y %b")

cpi_cpiu_data

,Year,Period,Label,Value,1-Month % Change
0,1998,M01,1998-01-01,162.000,0.1
1,1998,M02,1998-02-01,162.000,0.0
2,1998,M03,1998-03-01,162.000,0.0
3,1998,M04,1998-04-01,162.200,0.1
4,1998,M05,1998-05-01,162.600,0.2
...,...,...,...,...,...
306,2023,M07,2023-07-01,304.348,0.2
307,2023,M08,2023-08-01,306.269,0.6
308,2023,M09,2023-09-01,307.481,0.4
309,2023,M10,2023-10-01,307.619,0.0


In [26]:
# check data types
cpi_cpiu_data.dtypes

Year                         int64
Period                      object
Label               datetime64[ns]
Value                      float64
1-Month % Change           float64
dtype: object

In [27]:
# export as csv
cpi_cpiu_data.to_csv("cpi_cleaned_data.csv")

# Stocks Dataset Cleaning

In [28]:
# Import necessary packages
import pandas as pd 
import datetime as dt

import numpy as np
from pathlib import Path
%matplotlib inline


In [29]:
#Read and access individual stocks in S&P500
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp_table = pd.read_html(url)

In [30]:
sp_table[0]

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [31]:
sp_tickers = sp_table[0]['Symbol'].to_list()
sp_tickers

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CHRW',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA

In [32]:
#Pull stock information using yfinance API
df = yf.download(sp_tickers, start='2022-01-01', end='2022-12-31')['Adj Close']


[*********************100%%**********************]  503 of 503 completed

4 Failed downloads:
['KVUE', 'VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1641013200, endDate = 1672462800")
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-01-01 -> 2022-12-31)')
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [33]:
df

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,154.335785,18.750000,179.953888,124.095047,172.679993,133.418884,44.549999,394.007904,564.369995,170.609268,...,87.086357,63.911663,59.097824,55.229942,113.907425,131.481262,123.374718,583.900024,59.537766,230.225357
2022-01-04,149.118271,19.020000,177.669998,123.856819,170.800003,130.281097,45.130001,391.192200,554.000000,169.068359,...,86.808533,64.316154,61.320728,55.511478,115.115799,132.184280,124.626320,587.599976,61.233810,221.460236
2022-01-05,146.563782,18.680000,172.943970,124.507439,162.250000,129.695755,44.599998,384.303040,514.429993,166.458405,...,83.593773,64.946434,62.083389,54.618332,113.546867,130.508636,124.215492,558.179993,61.372837,213.039352
2022-01-06,147.076645,18.570000,170.056976,123.920967,159.750000,129.676559,44.860001,365.744873,514.119995,166.988083,...,83.980736,64.438446,63.543640,54.579494,113.390961,131.856842,123.183617,555.159973,63.893742,213.885391
2022-01-07,143.161026,19.280001,170.225037,123.600227,166.050003,130.079590,45.070000,358.729980,510.700012,162.606064,...,83.891434,65.002884,64.064499,54.754246,112.484695,130.344910,122.438377,530.859985,64.959557,207.658295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,147.932907,12.710000,131.127060,155.283295,85.250000,105.548416,63.380001,260.975525,338.450012,160.778229,...,80.091263,68.115936,105.091949,31.219862,108.321198,126.519440,125.714478,248.220001,46.140099,144.442429
2022-12-27,148.250107,12.530000,129.307236,155.178558,83.489998,105.928925,63.619999,260.210541,335.089996,159.168869,...,83.673149,68.730545,106.552086,31.455263,109.298492,127.500984,126.299927,251.000000,46.511505,143.986572
2022-12-28,146.802826,12.320000,125.339409,154.454987,82.489998,105.206932,62.599998,258.062653,328.329987,157.284744,...,79.466171,68.235489,104.801834,30.386156,107.541351,126.921875,125.019859,246.839996,45.682983,142.529861


In [34]:
# Identify any null values in the S&P 500 table
df.loc[:, df.isna().any()]

,BF.B,BRK.B,CEG,GEHC,KVUE,VLTO
Date,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-07,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2022-12-23,NaN,NaN,86.980545,57.408142,NaN,NaN
2022-12-27,NaN,NaN,86.585358,57.358223,NaN,NaN
2022-12-28,NaN,NaN,85.567749,57.408142,NaN,NaN


In [35]:
#Since yfinance is unable to recognize BF.B, deeper dive was done on yahoo finance which showed that yahoo finance had
#a different presentation of the stock symbol (BF-B)
yf.download('BF-B', start='2022-01-01', end='2022-12-31')['Adj Close']

[*********************100%%**********************]  1 of 1 completed


Date
2022-01-03    69.876350
2022-01-04    69.720177
2022-01-05    70.130119
2022-01-06    67.572845
2022-01-07    66.518684
                ...    
2022-12-23    65.477402
2022-12-27    65.704384
2022-12-28    64.707664
2022-12-29    65.280029
2022-12-30    64.816216
Name: Adj Close, Length: 251, dtype: float64

In [36]:
#Since yfinance is unable to recognize BRK.B, deeper dive was done on yahoo finance which showed that yfinance
# a different presentation of the stock symbol (BRK-B)
yf.download('BRK-B', start='2022-01-01', end='2022-12-31')['Adj Close']

[*********************100%%**********************]  1 of 1 completed


Date
2022-01-03    300.790009
2022-01-04    308.529999
2022-01-05    309.920013
2022-01-06    313.220001
2022-01-07    319.779999
                 ...    
2022-12-23    306.489990
2022-12-27    305.549988
2022-12-28    303.429993
2022-12-29    309.059998
2022-12-30    308.899994
Name: Adj Close, Length: 251, dtype: float64

In [37]:
# changing tickers to match yahoo finance ticker symbol
for i in range(len(sp_tickers)):
    if sp_tickers[i] == 'BRK.B':
        sp_tickers[i] = 'BRK-B'
    elif sp_tickers[i] == 'BF.B':
       sp_tickers[i]= 'BF-B'

In [38]:
df = sp_prices = yf.download(sp_tickers, start='2022-01-01', end='2022-12-31')['Adj Close']

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['KVUE', 'VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1641013200, endDate = 1672462800")


##### Note: KVUE was not included in the S&P 500 until May 2023, VLTO not until Oct 4, 2023

In [39]:
# Fill null values with zeros since KVUE and VLTO are missing some prices due to the difference in listed dates
df.loc[:, df.fillna(0).any()]

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,154.335785,18.750000,179.953888,124.095047,172.679993,133.418900,44.549999,394.007874,564.369995,170.609268,...,87.086349,63.911663,59.097820,55.229942,113.907433,131.481277,123.374718,583.900024,59.537762,230.225357
2022-01-04,149.118271,19.020000,177.669983,123.856812,170.800003,130.281097,45.130001,391.192261,554.000000,169.068344,...,86.808533,64.316162,61.320724,55.511486,115.115799,132.184296,124.626328,587.599976,61.233814,221.460205
2022-01-05,146.563766,18.680000,172.943985,124.507423,162.250000,129.695740,44.599998,384.303040,514.429993,166.458389,...,83.593765,64.946434,62.083401,54.618328,113.546883,130.508667,124.215492,558.179993,61.372833,213.039368
2022-01-06,147.076630,18.570000,170.056976,123.920944,159.750000,129.676544,44.860001,365.744904,514.119995,166.988098,...,83.980736,64.438454,63.543640,54.579502,113.390968,131.856888,123.183632,555.159973,63.893745,213.885376
2022-01-07,143.161041,19.280001,170.225067,123.600243,166.050003,130.079575,45.070000,358.729919,510.700012,162.606094,...,83.891434,65.002876,64.064491,54.754246,112.484695,130.344925,122.438370,530.859985,64.959557,207.658295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,147.932907,12.710000,131.127060,155.283310,85.250000,105.548424,63.380001,260.975555,338.450012,160.778229,...,80.091263,68.115936,105.091942,31.219860,108.321205,126.519447,125.714478,248.220001,46.140099,144.442413
2022-12-27,148.250137,12.530000,129.307236,155.178574,83.489998,105.928925,63.619999,260.210571,335.089996,159.168869,...,83.673149,68.730545,106.552094,31.455261,109.298485,127.500961,126.299927,251.000000,46.511501,143.986572
2022-12-28,146.802811,12.320000,125.339417,154.454987,82.489998,105.206924,62.599998,258.062622,328.329987,157.284760,...,79.466164,68.235489,104.801842,30.386158,107.541344,126.921883,125.019867,246.839996,45.682983,142.529846


In [40]:
#find percent change for each stock using the closing price
ClosingPrice = df.pct_change()
ClosingPrice

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,-0.033806,0.014400,-0.012692,-0.001920,-0.010887,-0.023518,0.013019,-0.007146,-0.018374,-0.009032,...,-0.003190,0.006329,0.037614,0.005098,0.010608,0.005347,0.010145,0.006337,0.028487,-0.038072
2022-01-05,-0.017131,-0.017876,-0.026600,0.005253,-0.050059,-0.004493,-0.011744,-0.017611,-0.071426,-0.015437,...,-0.037033,0.009800,0.012437,-0.016090,-0.013629,-0.012676,-0.003297,-0.050068,0.002270,-0.038024
2022-01-06,0.003499,-0.005889,-0.016693,-0.004710,-0.015408,-0.000148,0.005830,-0.048290,-0.000603,0.003182,...,0.004629,-0.007822,0.023521,-0.000711,-0.001373,0.010331,-0.008307,-0.005410,0.041075,0.003971
2022-01-07,-0.026623,0.038234,0.000988,-0.002588,0.039437,0.003108,0.004681,-0.019180,-0.006652,-0.026241,...,-0.001063,0.008759,0.008197,0.003202,-0.007992,-0.011467,-0.006050,-0.043771,0.016681,-0.029114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,0.001476,0.011943,-0.002798,-0.001041,0.004477,0.001389,0.008433,0.005023,0.005735,0.000550,...,-0.006401,0.012852,0.026445,0.011118,-0.000728,0.000621,0.001106,0.002869,0.003521,0.005033
2022-12-27,0.002144,-0.014162,-0.013878,-0.000674,-0.020645,0.003605,0.003787,-0.002931,-0.009928,-0.010010,...,0.044723,0.009023,0.013894,0.007540,0.009022,0.007758,0.004657,0.011200,0.008049,-0.003156
2022-12-28,-0.009763,-0.016760,-0.030685,-0.004663,-0.011977,-0.006816,-0.016033,-0.008255,-0.020174,-0.011837,...,-0.050279,-0.007203,-0.016426,-0.033988,-0.016077,-0.004542,-0.010135,-0.016574,-0.017813,-0.010117


In [41]:
ClosingPrice.to_csv("closing_price.csv")